In [ ]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()


# 현재 파일의 위치를 기준으로 프로젝트 루트(backend) 경로를 계산하여 추가
root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if root_path not in sys.path:
    sys.path.append(root_path)

true


In [3]:
from datetime import datetime

from fastapi import APIRouter, Depends, HTTPException, status
from sqlalchemy.orm import Session

from app.agents.multi_agent_graph import graph as multi_agent_graph
from app.deps import get_current_user, get_db
from app.models import Conversation, Message, User
from app.schemas import (
    ChatResponse,
    ConversationCreate,
    ConversationOut,
    MessageCreate,
    MessageOut,
)

router = APIRouter(prefix="/conversations", tags=["conversations"])

In [ ]:
def _get_conversation(db: Session, user_id: int, conversation_id: int) -> Conversation:

    conversation = (  # -> 채팅방
        db.query(Conversation)
        .filter(Conversation.id == conversation_id, Conversation.user_id == user_id)
        .first()  # -> 대화는 어차피 1개지만 객체로 받기 위해 추가
    )
    if not conversation:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND, detail="채팅방을 찾을 수 없습니다."
        )
    return conversation

In [ ]:
def get_all_messages(conversation_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user)):

    conversation = _get_conversation(db, current_user.id, conversation_id)

    all_messages = conversation.messages
    return all_messages

In [ ]:
@router.post("/{conversation_id}/messages", response_model=ChatResponse)
def create_message(
    conversation_id: int,
    payload: MessageCreate,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):

    conversation = _get_conversation(db, current_user.id, conversation_id)

    user_message = Message(
        conversation_id=conversation.id, role="user", content=payload.content
    )
    db.add(user_message)

    if not conversation.title or conversation.title == "새 대화":
        conversation.title = payload.content.strip()[:40]

    conversation.updated_at = datetime.utcnow()
    db.flush()

    try:
        all_messages = conversation.messages
        chat_history = []

        for msg in all_messages:
            if msg.id != user_message.id:
                chat_history.append({"role": msg.role, "content": msg.content})

        result = multi_agent_graph.invoke(
            {"query": payload.content, "chat_history": chat_history}
        )
        answer = result.get("answer") or "답변을 생성하지 못했습니다."

    except Exception as exc:
        db.rollback()
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail=f"Agent error: {exc}",
        ) from exc

    assistant_message = Message(
        conversation_id=conversation.id, role="assistant", content=answer
    )
    db.add(assistant_message)
    conversation.updated_at = datetime.utcnow()

    db.commit()
    db.refresh(conversation)
    db.refresh(user_message)
    db.refresh(assistant_message)

    return ChatResponse(
        conversation=ConversationOut.model_validate(conversation),
        user_message=MessageOut.model_validate(user_message),
        assistant_message=MessageOut.model_validate(assistant_message),
    )